In [1]:
import nrrd
import os
import pandas as pd
import numpy as np
from scipy.ndimage import zoom
import csv
import matplotlib.pyplot as plt
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, concatenate, Conv3DTranspose, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, load_model, save_model

In [2]:
data_path = 'F:\\Birva_Harsh\\NEW_KNEE_CT_100_V1\\Train'

In [3]:
knee_paths = []
mask_paths = []
csv_paths = []

for folder in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder)
    
    for file in os.listdir(folder_path):
        if file.endswith('.nrrd'):
            if 'segmentation' in file.lower():
                mask_path = os.path.join(folder_path, file)
                mask_paths.append(mask_path)
            else:
                knee_path = os.path.join(folder_path, file)
                knee_paths.append(knee_path)
                
        if file == 'LM_INFO_voxel_Coord_femur.csv':
            csv_path = os.path.join(folder_path, file)
            csv_paths.append(csv_path)

In [4]:
df_train = pd.DataFrame({'knee': knee_paths, 'mask': mask_paths})

In [5]:
data_path = 'F:\\Birva_Harsh\\NEW_KNEE_CT_100_V1\\Test'

In [6]:
knee_paths = []
mask_paths = []
csv_paths_test = []

for folder in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder)
    
    for file in os.listdir(folder_path):
        if file.endswith('.nrrd'):
            if 'segmentation' in file.lower():
                mask_path = os.path.join(folder_path, file)
                mask_paths.append(mask_path)
            else:
                knee_path = os.path.join(folder_path, file)
                knee_paths.append(knee_path)
                
        if file == 'LM_INFO_voxel_Coord_femur.csv':
            csv_path = os.path.join(folder_path, file)
            csv_paths_test.append(csv_path)

In [7]:
df_test = pd.DataFrame({'knee': knee_paths, 'mask': mask_paths})

In [8]:
df_test

,knee,mask
0,F:\Birva_Harsh\NEW_KNEE_CT_100_V1\Test\Shobhna...,F:\Birva_Harsh\NEW_KNEE_CT_100_V1\Test\Shobhna...
1,F:\Birva_Harsh\NEW_KNEE_CT_100_V1\Test\Soma__M...,F:\Birva_Harsh\NEW_KNEE_CT_100_V1\Test\Soma__M...
2,F:\Birva_Harsh\NEW_KNEE_CT_100_V1\Test\SUKHBIR...,F:\Birva_Harsh\NEW_KNEE_CT_100_V1\Test\SUKHBIR...
3,F:\Birva_Harsh\NEW_KNEE_CT_100_V1\Test\Sumangl...,F:\Birva_Harsh\NEW_KNEE_CT_100_V1\Test\Sumangl...
4,F:\Birva_Harsh\NEW_KNEE_CT_100_V1\Test\Sumangl...,F:\Birva_Harsh\NEW_KNEE_CT_100_V1\Test\Sumangl...
5,F:\Birva_Harsh\NEW_KNEE_CT_100_V1\Test\Suman_D...,F:\Birva_Harsh\NEW_KNEE_CT_100_V1\Test\Suman_D...
6,F:\Birva_Harsh\NEW_KNEE_CT_100_V1\Test\SUNITA ...,F:\Birva_Harsh\NEW_KNEE_CT_100_V1\Test\SUNITA ...
7,F:\Birva_Harsh\NEW_KNEE_CT_100_V1\Test\SUSHMA ...,F:\Birva_Harsh\NEW_KNEE_CT_100_V1\Test\SUSHMA ...
8,F:\Birva_Harsh\NEW_KNEE_CT_100_V1\Test\Sushma_...,F:\Birva_Harsh\NEW_KNEE_CT_100_V1\Test\Sushma_...
9,F:\Birva_Harsh\NEW_KNEE_CT_100_V1\Test\Tajinde...,F:\Birva_Harsh\NEW_KNEE_CT_100_V1\Test\Tajinde...


# Preprocessing

In [12]:
x,y=nrrd.read(df_test.loc[7,'knee'])

In [13]:
y

OrderedDict([('type', 'int'),
             ('dimension', 3),
             ('space', 'left-posterior-superior'),
             ('sizes', array([512, 512, 817])),
             ('space directions',
              array([[0.976562  , 0.        , 0.        ],
                     [0.        , 0.976562  , 0.        ],
                     [0.        , 0.        , 0.99999999]])),
             ('kinds', ['domain', 'domain', 'domain']),
             ('endian', 'little'),
             ('encoding', 'gzip'),
             ('space origin', array([-241.      ,  -80.5     , -877.312988]))])

In [9]:
def read_csv_to_dict_LM_INFO(csv_file):
    LM_INFO = {}
    with open(csv_file, mode='r') as infile:
        reader = csv.reader(infile)       
        for rows in reader:
            LM_INFO[rows[0]] = np.asarray([float(rows[1]), float(rows[2]), float(rows[3])])
            #print(rows[0], ':', float(rows[1]), float(rows[2]), float(rows[3]))
            
    return LM_INFO

In [10]:
def zoom_1mm(img,original_space,new_space=[1.0,1.0,1.0]):
    zoom_factor = [original_space[i] / new_space[i] for i in range(3)]
    zoomed_image = zoom(img, zoom_factor, order=0)
    
    return zoomed_image

In [11]:
def knee_centre(path, original_space):
    KC = read_csv_to_dict_LM_INFO(path)['FKC']
    KC_1mm = [round(original_space[i] * KC[i]) for i in range(3)]
    
    return KC_1mm

In [12]:
def CROP_3D_CT_IMAGE_AT_GIVEN_POINT_NEW_256(img, centered_at, crop_size):
    crop_size = np.asanyarray(crop_size)
    img = np.pad(img, ((128,128),(128,128),(128,128)),'edge')   

    sp_idx = np.asarray(128+centered_at-crop_size//2,dtype=int)
    ep_idx = np.asarray(128+centered_at+crop_size//2,dtype=int)
       
    return img[sp_idx[0]:ep_idx[0], sp_idx[1]:ep_idx[1], sp_idx[2]:ep_idx[2]]

In [13]:
def crop_KC(path, csv_path):
    img, info = nrrd.read(path)
    spc_dir = np.diag(info['space directions'])

    img_1mm = zoom_1mm(img, spc_dir)
    KC = knee_centre(csv_paths[i], spc_dir)
    
    info['sizes'] = np.array([256.0,256.0,256.0])
    info['space directions'] = np.eye(3)
   

In [16]:
crop_knee_train = 'F:\\Birva_Harsh\\Knee_Crop_Data\\train\\Knee'
crop_mask_train = 'F:\\Birva_Harsh\\Knee_Crop_Data\\train\\Mask'

In [17]:
for i in range(len(df_train)):
    img, kinfo = nrrd.read(df_train.loc[i,'knee'])
    mask, minfo = nrrd.read(df_train.loc[i,'mask'])
    
    spc_dir = np.diag(kinfo['space directions'])
    
    img_1mm = zoom_1mm(img, spc_dir)
    mask_1mm = zoom_1mm(mask, spc_dir)
    
    KC = knee_centre(csv_paths[i], spc_dir)
    
    kinfo['sizes'] = np.array([256.0,256.0,256.0])
    minfo['sizes'] = np.array([256.0,256.0,256.0])
    kinfo['space directions'] = np.eye(3)
    minfo['space directions'] = np.eye(3)
    
    for j in range(5):
        shift = np.random.randint(-30,30,size=3)
        shift_KC = KC - shift
        
        crop_knee = CROP_3D_CT_IMAGE_AT_GIVEN_POINT_NEW_256(img_1mm, shift_KC, [256,256,256])
        crop_mask = CROP_3D_CT_IMAGE_AT_GIVEN_POINT_NEW_256(mask_1mm, shift_KC, [256,256,256])
        
        output_img_path = os.path.join(crop_knee_train, f"{os.path.basename(os.path.dirname(df_train.loc[i,'knee']))}_{j+1}.nrrd")
        output_mask_path = os.path.join(crop_mask_train, f"{os.path.basename(os.path.dirname(df_train.loc[i,'knee']))}_{j+1}.nrrd")
        
        nrrd.write(output_img_path, crop_knee, kinfo)
        nrrd.write(output_mask_path, crop_mask, minfo)
        
        print(f"{os.path.basename(os.path.dirname(df_train.loc[i,'knee']))}_{j+1}")

Abhijit Banerjeer_1
Abhijit Banerjeer_2
Abhijit Banerjeer_3
Abhijit Banerjeer_4
Abhijit Banerjeer_5
Amar_Bahadur_Singh_70_M_1
Amar_Bahadur_Singh_70_M_2
Amar_Bahadur_Singh_70_M_3
Amar_Bahadur_Singh_70_M_4
Amar_Bahadur_Singh_70_M_5
AminChand Mehta_1
AminChand Mehta_2
AminChand Mehta_3
AminChand Mehta_4
AminChand Mehta_5
Amita_Pandey_62_F_1
Amita_Pandey_62_F_2
Amita_Pandey_62_F_3
Amita_Pandey_62_F_4
Amita_Pandey_62_F_5
Anita Chanan_1
Anita Chanan_2
Anita Chanan_3
Anita Chanan_4
Anita Chanan_5
Archana_Sawhney_57_F_1
Archana_Sawhney_57_F_2
Archana_Sawhney_57_F_3
Archana_Sawhney_57_F_4
Archana_Sawhney_57_F_5
Ashokkumar_Shah_71YM_1
Ashokkumar_Shah_71YM_2
Ashokkumar_Shah_71YM_3
Ashokkumar_Shah_71YM_4
Ashokkumar_Shah_71YM_5
Ashutosh_Tiwari_69_M_1
Ashutosh_Tiwari_69_M_2
Ashutosh_Tiwari_69_M_3
Ashutosh_Tiwari_69_M_4
Ashutosh_Tiwari_69_M_5
BALA DEVI RT KNEE_1
BALA DEVI RT KNEE_2
BALA DEVI RT KNEE_3
BALA DEVI RT KNEE_4
BALA DEVI RT KNEE_5
Balwinder Kaur_1
Balwinder Kaur_2
Balwinder Kaur_3
Balwinder

In [23]:
crop_knee_test = 'F:\\Birva_Harsh\\Knee_Crop_Data\\test\\Knee'
crop_mask_test = 'F:\\Birva_Harsh\\Knee_Crop_Data\\test\\Mask'

In [24]:
for i in range(len(df_test)):
    img, kinfo = nrrd.read(df_test.loc[i,'knee'])
    mask, minfo = nrrd.read(df_test.loc[i,'mask'])
    
    spc_dir = np.diag(kinfo['space directions'])
    
    img_1mm = zoom_1mm(img, spc_dir)
    mask_1mm = zoom_1mm(mask, spc_dir)
    
    KC = knee_centre(csv_paths_test[i], spc_dir)
    
    kinfo['sizes'] = np.array([256.0,256.0,256.0])
    minfo['sizes'] = np.array([256.0,256.0,256.0])
    kinfo['space directions'] = np.eye(3)
    minfo['space directions'] = np.eye(3)
    
    for j in range(5):
        shift = np.random.randint(-30,30,size=3)
        shift_KC = KC - shift
        
        crop_knee = CROP_3D_CT_IMAGE_AT_GIVEN_POINT_NEW_256(img_1mm, shift_KC, [256,256,256])
        crop_mask = CROP_3D_CT_IMAGE_AT_GIVEN_POINT_NEW_256(mask_1mm, shift_KC, [256,256,256])
        
        output_img_path = os.path.join(crop_knee_test, f"{os.path.basename(os.path.dirname(df_test.loc[i,'knee']))}_{j+1}.nrrd")
        output_mask_path = os.path.join(crop_mask_test, f"{os.path.basename(os.path.dirname(df_test.loc[i,'knee']))}_{j+1}.nrrd")
        
        nrrd.write(output_img_path, crop_knee, kinfo)
        nrrd.write(output_mask_path, crop_mask, minfo)
        
        print(f"{os.path.basename(os.path.dirname(df_test.loc[i,'knee']))}_{j+1}")

Shobhnaben_H_Shah_F74Y_1
Shobhnaben_H_Shah_F74Y_2
Shobhnaben_H_Shah_F74Y_3
Shobhnaben_H_Shah_F74Y_4
Shobhnaben_H_Shah_F74Y_5
Soma__Mondal_1
Soma__Mondal_2
Soma__Mondal_3
Soma__Mondal_4
Soma__Mondal_5
SUKHBIR KAUR LT KNEE_1
SUKHBIR KAUR LT KNEE_2
SUKHBIR KAUR LT KNEE_3
SUKHBIR KAUR LT KNEE_4
SUKHBIR KAUR LT KNEE_5
Sumangla naithani lt_1
Sumangla naithani lt_2
Sumangla naithani lt_3
Sumangla naithani lt_4
Sumangla naithani lt_5
Sumangla_Naithani rt_1
Sumangla_Naithani rt_2
Sumangla_Naithani rt_3
Sumangla_Naithani rt_4
Sumangla_Naithani rt_5
Suman_Devi_50_F_1
Suman_Devi_50_F_2
Suman_Devi_50_F_3
Suman_Devi_50_F_4
Suman_Devi_50_F_5
SUNITA KULAKARNI LT KNEE_1
SUNITA KULAKARNI LT KNEE_2
SUNITA KULAKARNI LT KNEE_3
SUNITA KULAKARNI LT KNEE_4
SUNITA KULAKARNI LT KNEE_5
SUSHMA MAGO LT KNEE_1
SUSHMA MAGO LT KNEE_2
SUSHMA MAGO LT KNEE_3
SUSHMA MAGO LT KNEE_4
SUSHMA MAGO LT KNEE_5
Sushma_Sharma_82_F_1
Sushma_Sharma_82_F_2
Sushma_Sharma_82_F_3
Sushma_Sharma_82_F_4
Sushma_Sharma_82_F_5
Tajinder_Kaur_5

In [ ]:
def plot_3d(img,mask):
    plt.figure(figsize=(15,8))
    plt.subplot(2, 3, 1)
    plt.imshow(img[128,:,:].T,cmap='gray')
    plt.axis('off')
    plt.title("X")
    
    plt.subplot(2, 3, 2)
    plt.imshow(img[:,128,:].T,cmap='gray')
    plt.axis('off')
    plt.title("Y")
    
    plt.subplot(2, 3, 3)
    plt.imshow(img[:,:,128].T,cmap='gray')
    plt.axis('off')
    plt.title("Z")
    
    
    plt.subplot(2, 3, 4)
    plt.imshow(mask[128,:,:].T,interpolation='nearest')
    plt.axis('off')
    plt.title("Mask X")
    
    plt.subplot(2, 3, 5)
    plt.imshow(mask[:,128,:].T,interpolation='nearest')
    plt.axis('off')
    plt.title("Mask Y")
    
    plt.subplot(2, 3, 6)
    plt.imshow(mask[:,:,128].T,interpolation='nearest')
    plt.axis('off')
    plt.title("Mask Z")
plt.show()

In [ ]:
img, info = nrrd.read(f"G:\Harsh_Birva_minor_Proj\Knee_Crop_Data\Knee\91_2.nrrd")
mask, info = nrrd.read(f"G:\Harsh_Birva_minor_Proj\Knee_Crop_Data\Mask\91_2.nrrd")

In [ ]:
plot_3d(img,mask)

In [ ]:
img, info = nrrd.read("D:\\NEW_KNEE_CT_DATA\\NEW_KNEE_CT_100_V1\\Kalparatnashriji_Ms_F70Y\\5303 lt.nrrd")
mask, minfo = nrrd.read("D:\\NEW_KNEE_CT_DATA\\NEW_KNEE_CT_100_V1\\Kalparatnashriji_Ms_F70Y\\Segmentation.seg.nrrd")

In [ ]:
spc_dir = np.diag(info['space directions'])

In [ ]:
zoom_factor = [spc_dir[i] for i in range(3)]

In [ ]:
img_1mm = zoom(img,spc_dir, order=0)

In [ ]:
img_1mm.shape

In [ ]:
mask_1mm = zoom(mask,spc_dir, order=0)

In [ ]:
KC = knee_centre("D:\\NEW_KNEE_CT_DATA\\NEW_KNEE_CT_100_V1\\Kalparatnashriji_Ms_F70Y\\LM_INFO_voxel_Coord_femur.csv", spc_dir)

In [ ]:
KC = np.array(KC)

In [ ]:
crop_knee = CROP_3D_CT_IMAGE_AT_GIVEN_POINT_NEW_256(img_1mm, KC, [256,256,256])
crop_mask = CROP_3D_CT_IMAGE_AT_GIVEN_POINT_NEW_256(mask_1mm,KC, [256,256,256])

In [ ]:
plot_3d(crop_knee,crop_mask)

# Training

In [ ]:
knee_files = glob(pathname = crop_knee_path+'\\*')
knee_files[:1]

In [ ]:
mask_files = [path.replace('\\Knee\\', '\\Mask\\') for path in knee_files]
mask_files[:1]

In [ ]:
df_knee = pd.DataFrame( data = {
    'knee': knee_files,
    'mask': mask_files
})

In [ ]:
df_train, df_test = train_test_split(df_knee, random_state=11, test_size=0.205)

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
def load_and_preprocess_image(file_path,resize):
    img,info = nrrd.read(file_path)
    if resize == 128:
        img = img[::2,::2,::2]
    img = np.clip(img, a_min=-1024, a_max=None)
    img = img[:,:,:,np.newaxis]   
    img_scaled = img.astype(np.float32) / 1024.0
    return img_scaled

In [ ]:
def load_and_preprocess_mask(file_path,resize):
    img,info = nrrd.read(file_path)
    if resize == 128:
        img = img[::2,::2,::2]
    
    img_1 = np.where(img == 1, 1, 0)
    img_2 = np.where(img == 2, 1, 0)
    img_3 = np.where(img == 3, 1, 0)
    
    mask = np.stack((img_1, img_2, img_3), axis=-1)
    mask = mask.astype(np.float32)
    return mask

In [ ]:
load_and_preprocess_mask(df_train.loc[0,'mask'],128)

In [ ]:
def generate_data_generator(dataframe,resize, batch_size=1):
    num_samples = len(dataframe)
    while True:
        indices = np.random.randint(0, num_samples, batch_size)
        batch_knee = []
        batch_mask = []
        for idx in indices:
            knee_img = load_and_preprocess_image(dataframe.iloc[idx]['knee'],resize)
            mask_img = load_and_preprocess_mask(dataframe.iloc[idx]['mask'],resize)
            batch_knee.append(knee_img)
            batch_mask.append(mask_img)
        batch_knee = np.array(batch_knee)
        batch_mask = np.array(batch_mask)
        
        yield batch_knee, batch_mask

In [ ]:
batch_size = 1
train_generator = generate_data_generator(df_train, batch_size=batch_size,resize=128)
test_generator = generate_data_generator(df_test, batch_size=batch_size,resize=128)

In [ ]:
def dice_coeff(y_true, y_pred, smooth=100):
    y_true_flatten = K.flatten(y_true)
    y_pred_flatten = K.flatten(y_pred)

    intersection = K.sum(y_true_flatten * y_pred_flatten)
    union = K.sum(y_true_flatten) + K.sum(y_pred_flatten)
    return (2 * intersection + smooth) / (union + smooth)

def dice_coeff_loss(y_true, y_pred, smooth=100):
    return 1 - dice_coeff(y_true, y_pred, smooth)

In [ ]:
def encoder_block(inputs, filters):
    conv = Conv3D(filters, kernel_size = (3,3,3), padding="same", activation = 'relu', kernel_initializer='he_uniform')(inputs)
    conv = Conv3D(filters, kernel_size = (3,3,3), padding="same", activation = 'relu', kernel_initializer='he_uniform')(conv)
    x = BatchNormalization()(conv)
    pool = MaxPooling3D(pool_size=(2,2,2), padding="same")(x)
    
    return pool,x

def decoder_block(inputs, filters, x):
    up = Conv3DTranspose(filters, kernel_size=(3,3,3), strides=(2, 2, 2), padding="same", kernel_initializer='he_uniform')(inputs)
    con = concatenate([up, x])
    conv = Conv3D(filters, kernel_size = (3,3,3), padding="same", activation = 'relu', kernel_initializer='he_uniform')(con)
    conv = Conv3D(filters, kernel_size = (3,3,3), padding="same", activation = 'relu', kernel_initializer='he_uniform')(conv)
    x = BatchNormalization()(conv)
    
    return x

In [ ]:
f = 128

In [ ]:
def Pocketnet_32(input_shape = (None, None, None, 1)):
    inputs = Input(input_shape)
    x = Conv3D(filters = f, kernel_size = (3,3,3), padding = 'same', activation = 'relu', kernel_initializer='he_uniform')(inputs)
    x = BatchNormalization()(x)
    
    b1,c1 = encoder_block(inputs, f)
    b2,c2 = encoder_block(b1, f)
    b3,c3 = encoder_block(b2, f)
    b4,c4 = encoder_block(b3, f)

    conv = Conv3D(f, kernel_size = (3,3,3), padding="same", activation = 'relu', kernel_initializer='he_uniform')(b4)
    conv = Conv3D(f, kernel_size = (3,3,3), padding="same", activation = 'relu', kernel_initializer='he_uniform')(conv)

    b4 = decoder_block(conv, f, c4)
    b5 = decoder_block(b4, f, c3)
    b6 = decoder_block(b5, f, c2)
    b7 = decoder_block(b6, f, c1)
    
    outputs = Conv3D(filters = 3, kernel_size = (1,1,1), activation = "sigmoid")(x)
    
    return Model(inputs=[inputs], outputs = [outputs])

In [ ]:
filepath = "G:\\Harsh_Birva_minor_Proj\\Knee_models\\pocketnet_32.keras"

checkpoint = ModelCheckpoint(filepath, 
                             verbose=1 
                             )

In [ ]:
pocket_32 = Pocketnet_32(input_shape = (None,None,None,1))
pocket_32.compile(optimizer='adam', loss=dice_coeff_loss, metrics=[dice_coeff])

In [ ]:
epochs = 50

In [ ]:
history = pocket_32.fit(
        train_generator,
        steps_per_epoch=len(df_train) // batch_size,
        epochs=epochs,
        callbacks=[checkpoint],
        validation_data=test_generator,
        validation_steps=len(df_test) // batch_size
    )

In [ ]:
img_1mm